In [1]:
# ! pip install jugaad_data, nsetools
from jugaad_data.nse import stock_df
from nsetools import Nse

from datetime import date, datetime, timedelta
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import mplfinance as fplt
import plotly.graph_objects as go
from plotly import colors as plotly_colors

from multiprocessing import Pool

import requests
from bs4 import BeautifulSoup
import random

from os import listdir
from os.path import isfile, join

import json
import warnings
import math
from random import sample

In [98]:
def get_stocks(name):
    try:
        df = stock_df(symbol=name, from_date = current_date - timedelta(days = 465), to_date = current_date, series="EQ").drop(drop,axis=1)
        ID, NAME, _ = all_stocks[name].split('_')
        save = f"./data/{ID}_{NAME}_{str(current_date)}.csv"
        df.to_csv(save,index=None)
    except Exception as e:
        print(e)
        pass


 
path = './data'
AS = AnalyseStocks(check_fresh = False)
all_stocks = AS.all_stocks

drop = ['SERIES','PREV. CLOSE','VWAP','VOLUME','VALUE','NO OF TRADES']
current_date = date.today()
stocks = set(all_stocks.keys()) - set([i.split('_')[0] for i in listdir('./data')]) 




pool = Pool(4)
results = pool.map(get_stocks,stocks)
pool.close()
pool.join()

# Code

In [2]:
class DataHandler:
    def __init__(self, data_path = './data', check_fresh = True):
        self.present = date.today()
        self.week_num = self.present.strftime("%W")
        
        self.data_path = data_path
        
        if check_fresh:
            self.__fresh()
        self.data = self.read_data()
        self.all_stocks = self.read_data()['all_stocks']
        
         
    def __fresh(self,):
        files = listdir(self.data_path)
        if not len(files):
            raise Exception(f"No CSV data files present at {self.data_path} Download new data for analysis")

        self.data = self.read_data()
        for file in files:
            key, _ , _ = file.split('_')
            self.data['all_stocks'][key] = file
        self.update_data(self.data)
  
    
    def read_data(self, path = './', file = 'data.json'):
        '''
        Write the data in json file
        args:
            path: Path of the directory
            File: Json Filename
        '''
        with open(join(path,file)) as f:
            return json.load(f)


    def update_data(self, updated_data:dict, path:str = './', file:str = 'data.json'):
        '''
        Update the data in the json file
        args:
            updated_data: Dictonary you want to update
            path: Path of the directory
            File: Json Filename
        '''
        with open(join(path,file), 'w') as f:
            json.dump(updated_data,f)
            return True 
    
    
    def open_live_stock_data(self,name:str):
        '''
        Open the fresh stock from the market
        args:
            name: ID of the stock given
        '''
        drop = ['SERIES','PREV. CLOSE','VWAP','VOLUME','VALUE','NO OF TRADES']
        return stock_df(symbol=name, from_date = self.present - timedelta(days = 465), to_date = self.present, series="EQ").drop(drop,axis=1)
    
    
    def open_downloaded_stock(self,name:str):
        '''
        Open the Individual stock based on it's Official Term
        args:
            name: Name / ID given to the stock. Example, Infosys is "INFY"
        returns: DataFrame of that stock
        '''
        return pd.read_csv(join(self.data_path,self.all_stocks[name]))

In [40]:
class AnalyseStocks(DataHandler):
    def __init__(self, check_fresh = True):
        '''
        args:
            path: Path where all the stock files are saved
        '''
        super().__init__(check_fresh = check_fresh)
        self.eligible = {}
        self.registered_stocks = self.read_data()['registered_stocks']
        self.colors = self.read_data()['colors']
        self.rising = {}
    

    def is_eligible(self, df, mv = 44, names:tuple = ('DATE','OPEN','CLOSE','LOW','HIGH')):
        '''
        Find the Positive Stocks which are about to rise on the Moving average line
        args:
            df: DataFrame
            mv: Moving Average to Consider
            names: Tuple of column names showing ('DATE','OPEN','CLOSE','LOW','HIGH')
        '''
        Date, Open, Close, Low, High = names
        Average = f'{str(mv)}-SMA'
        
        stocks = df.sort_index(ascending=False,) # Sort the values else Moving average for new values will be empty
        stocks[Average] = stocks[Close].rolling(mv, min_periods = 1).mean()
        
        last_traded = stocks.iloc[-1,:]
        low, high, open_ , close, avg, symbol = last_traded[Low],last_traded[High],last_traded[Open],last_traded[Close], last_traded[Average], last_traded['SYMBOL']
        
        if high < avg or close < open_: # if red candle or below Average Line, Discard
            return False
        

        if (abs(low - avg) <= 13) or (abs(high - avg) <= 13 or (abs(close - avg) < 13)):
            self.rising[symbol] = stocks.iloc[-6:-1,-1].mean() < avg  # whether the moving average is Upward or Downward
            return {symbol : round(low - avg,2)}

        return False
    
    
    def update_eligible(self):
        '''
        Save all Eligible stocks for the current week
        '''
        for key in self.registered_stocks:
            result = self.is_eligible(self.open_downloaded_stock(key))
            if result:
                self.eligible.update(result)
        return self.eligible
            
    
    def plot_candlesticks(self,df, names = ('DATE','OPEN','CLOSE','LOW','HIGH'), mv:list = [44,100,200]):
        '''
        Plot a candlestick on a given dataframe
        args:
            df: DataFrame
            names: Tuple of column names showing ('DATE','OPEN','CLOSE','LOW','HIGH')
            mv: Moving Averages
        '''
        stocks = df.copy()
        Date, Open, Close, Low, High = names
        colors = sample(self.colors,len(mv))
        stocks.sort_index(ascending=False, inplace = True)  # Without reverse, recent rolling mean will be either NaN or equal to the exact value
    

        candle = go.Figure(data = [go.Candlestick(x = stocks[Date], name = 'Trade',
                                                       open = stocks[Open], 
                                                       high = stocks[High], 
                                                       low = stocks[Low], 
                                                       close = stocks[Close]),])
        for i in range(len(mv)):
            stocks[f'{str(mv[i])}-SMA'] = stocks[Close].rolling(mv[i], min_periods = 1).mean()
            candle.add_trace(go.Scatter(name=f'{str(mv[i])} MA',x=stocks[Date], y=stocks[f'{str(mv[i])}-SMA'], 
                                             line=dict(color=colors[i], width=1.1)))

        candle.update_xaxes(
            title_text = 'Date',
            rangeslider_visible = True,
            rangeselector = dict(
                buttons = list([
                    dict(count = 1, label = '1M', step = 'month', stepmode = 'backward'),
                    dict(count = 6, label = '6M', step = 'month', stepmode = 'backward'),
                    dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
                    dict(count = 1, label = '1Y', step = 'year', stepmode = 'backward'),
                    dict(step = 'all')])))

        candle.update_layout(autosize = False, width = 1400, height = 600,
                             title = {'text': f"{stocks['SYMBOL'][0]} | {self.all_stocks[stocks['SYMBOL'][0]]}",'y':0.97,'x':0.5,
                                      'xanchor': 'center','yanchor': 'top'},
                             margin=dict(l=30,r=30,b=30,t=30,pad=2),
                             paper_bgcolor="lightsteelblue",)

        candle.update_yaxes(title_text = 'Price in Rupees', tickprefix = u"\u20B9" ) # Rupee symbol
        candle.show()

# Investing

In [41]:
class Investing(AnalyseStocks):
    def __init__(self,rolling_mean:int=44):
        '''
        args:
            rolling_mean: Rolling Simple Mean to calulate
        '''
        super().__init__()
        self.rm = rolling_mean
        self._eligible = self.update_eligible()
        self.data = self.read_data()
        
    
    def pick(self,budget, High = 'HIGH', Close = 'CLOSE', delta:float=0.01, nifty:str = 'nifty_500'):
        '''
        Pick Stocks based on all available and which are within your budget
        args:
            budget: Total available budget. Stocks under this budget will be considered only
            High: Column name which show High
            Close: Column Name which shows last closing price
            delta: Value above the Last Highest Traded Price
            nifty: nifty index to consider
        ''' 
        keys = set(self._eligible.keys()).intersection(set(self.data[nifty])) if nifty else list(self._eligible.keys()) 
        values = []
        for key in keys:
            df = self.open_downloaded_stock(key)
            if df.loc[0,High] + delta > budget:# or df.loc[0,Close] < 50:
                del self._eligible[key]
                
            else:
                values.append(df.iloc[0,:])
                
        columns = df.columns
        df = pd.DataFrame(values,columns=columns,index = range(len(values)))
        df = df.merge(pd.DataFrame({'SYMBOL':self._eligible.keys(), 'Diff':self._eligible.values()}),on='SYMBOL')
        self.picked = df.sort_values('Diff',ascending = True)
        self.picked['Rising'] = self.picked['SYMBOL'].apply(lambda x: self.rising[x])
        return self.picked
    
    
    def show_particulars(self, df, budget:float, max_loss_capacity:float, risk_to_reward_ratio:float=2, Low:str = 'LOW', High:str = 'HIGH', delta:float = 0.3, plot_candle:bool = False):
        '''
        Display the particulars of a trade before buying
        args:
            df: DtaFrame of a particular stock
            loss_capacity: How much loss you can survive at the end of day PER SHARE. Total capacity will be No of shares * per share loss capacity
            risk_to_reward_ratio: How much profit you want to have. It is twice of loss_capacity per share for 44 Moving average
            budget : How much you have for investing purpose
            Low: Column name which describes LOW of the previous trade
            High =  Column name which describes High of the previous trade
            delta: A min amount above which you'll buy
            plot_candle: Plot the candlestick for the stock
        '''
        if risk_to_reward_ratio > 2:
            warnings.warn(f"Don't be greedy with risk to reward ratio of {risk_to_reward_ratio}. Stick to system")
            
        if max_loss_capacity > 0.10 * budget:
            warnings.warn(f"You are risking {round(max_loss_capacity/budget,2)*100}% of your total budget. Try keeping it less than 10% @ Rs{round(budget/10,2)}")
            
        risk = max_loss_capacity # per share
        entry = df.loc[0,High] + delta # Last Day MAX + Delta
        stop_loss = min(df.loc[:2,Low].values) # Min of the last 2 
        max_loss = entry - stop_loss
        stop_loss_perc = round((max_loss / entry) *100,2)
        diff = entry - stop_loss
        quantity = min(risk // diff , budget // entry)
        profit = risk_to_reward_ratio * diff
        profit_perc = round((profit/entry)*100,2)
        target = round(entry + profit,2)
        
        
        if plot_candle:
            AnalyseStocks().plot_candlesticks(df)
    
        if budget < entry:
            warnings.warn(f"Budget for this stock should be a minimum of Rs. {entry}")
            return None
        
        if quantity < 1:
            r = round(risk + (diff - risk), 2)
            warnings.warn(f"Risk should be atleast {r} for you to afford for this stock")
            return None
            
        return {'Buying Price':round(entry,2),'Stop-Loss %': stop_loss_perc,'Target %':profit_perc,'Quantity':quantity,'Stop-Loss Price':stop_loss,'Trigger Price':target,'Risk Per Share':round(diff,2),
                'Profit Per Share':round(profit,2),'Max loss on this config':round(quantity*diff,2),'Max Gain on this config': round(quantity*profit,2),'Price To Earning Ratio': round(entry / (diff*2 ),2)}
    
In = Investing()

In [46]:
picked = In.pick(1000,nifty='nifty_200')
picked.head(60)

,DATE,OPEN,HIGH,LOW,LTP,CLOSE,52W H,52W L,SYMBOL,Diff,Rising
5,2021-07-23,392.00,401.55,387.30,397.25,398.15,487.75,363.25,BIOCON,-10.12,True
1,2021-07-23,749.00,764.00,740.10,755.30,755.40,799.00,400.15,AXISBANK,-8.76,True
11,2021-07-23,422.90,429.95,419.50,428.50,428.90,441.95,175.50,SBIN,-6.01,True
14,2021-07-23,263.90,270.25,263.25,267.55,267.80,296.30,91.20,VEDL,-4.26,False
10,2021-07-23,397.70,400.55,394.00,399.00,397.70,501.70,159.60,JINDALSTEL,-3.71,False
4,2021-07-23,84.95,87.00,83.70,85.50,85.40,92.50,45.35,FEDERALBNK,-2.75,True
15,2021-07-23,867.00,882.20,867.00,868.00,868.95,956.00,682.00,COROMANDEL,-1.73,True
16,2021-07-23,124.30,125.05,123.45,124.70,124.30,138.90,48.00,ASHOKLEY,-1.12,False
12,2021-07-23,207.00,213.60,206.50,212.45,212.40,239.20,163.35,ITC,-0.50,True
2,2021-07-23,143.50,144.55,142.80,143.90,143.75,154.90,104.00,CASTROLIND,-0.44,True


In [ ]:
def suggest(total_budget, loss_threshold, names):
    total_bearable_loss = loss_threshold * total_budget # How much % of your money, if Each trade goes wrong, you can bear
    ideal_risk = total_bearable_loss / len(names) # Ideally risk should be equally distributed in every stock
    

Total Budget = 7000
Total Risk = 700
Ideal Risk = 700 / No of available  => 700 / 13
Profit = Always twice of Risk

A: {'Buying Price': 982.25,'Risk Per Share': 12.9}
B: {'Buying Price': 883.75,'Risk Per Share': 31.05}
C: {'Buying Price': 830.25,'Risk Per Share': 26.5}
D: {'Buying Price': 678.3,'Risk Per Share': 29.0}
E: {'Buying Price': 660.8,'Risk Per Share': 21.7}
F: {'Buying Price': 552.4,'Risk Per Share': 21.4}
G: {'Buying Price': 538.3,'Risk Per Share': 13.45}
H: {'Buying Price': 432.3,'Risk Per Share': 23.8}
I: {'Buying Price': 241.1,'Risk Per Share': 9.35}
J: {'Buying Price': 222.0,'Risk Per Share': 16.9}
K: {'Buying Price': 147.65,'Risk Per Share': 5.65}
L: {'Buying Price': 143.4,'Risk Per Share': 2.85}
M: {'Buying Price': 29.75,'Risk Per Share': 0.8,}


Scenario 1:

Allocation = 7 * M
Total Risk = (800 * 29.75) => 640
Total Profit = 640 * 2 => 1280
No of Allocated = 1 {M}
Overall Risk = 1280 / 1


Scenario 2:

Allocation = (7 * B) + (1 * D) + ( 4 * M) => (7* 883.75) + (1* 678.3) + (4*29.75)
Total Risk = (7 * 31.05) + ( 1 * 29 ) + (4*0.8) => 249.549
Total Profit = 249.549 * 2 => 499.09
No of Allocated = 3 {B,D,M}
Overall risk = 499 / 3


Scenario 3:

Allocation = (7 * B) + (28*M) => (7* 883.75) + (28 * 29.75)
Total Risk = (7 * 31.05) + (28*0.8) => 239.75
Total Profit = 2 * 239.75 => 479.5
Total Allocation = 2 {B,M}
Overall risk = 479 / 2

In [ ]:
def is_eligible(self, df, mv = 44, names:tuple = ('DATE','OPEN','CLOSE','LOW','HIGH')):
    '''
    Find the Positive Stocks which are about to rise on the Moving average line
    args:
        df: DataFrame
        mv: Moving Average to Consider
        names: Tuple of column names showing ('DATE','OPEN','CLOSE','LOW','HIGH')
    '''
    Date, Open, Close, Low, High = names
    Average = f'{str(mv)}-SMA'

    stocks = df.sort_index(ascending=False,) # Sort the values else Moving average for new values will be empty
    stocks[Average] = stocks[Close].rolling(mv, min_periods = 1).mean()

    last_traded = stocks.iloc[-1,:]
    if (last_traded[Close] > last_traded[Open]) and (last_traded[High] > last_traded[Average]) and (-15 <= (last_traded[Low] - last_traded[Average]) <= last_traded[High] - last_traded[Low]):
        return {last_traded['SYMBOL'] : round(last_traded[Low] - last_traded[Average],2)}

    return False